Tipos de datos complejos
===


In [0]:
%load_ext bigdata
%pig_init

## Datos simples

Los siguientes son los tipos de datos soportados por Pig:

     int      long      float       double      chararray  
     boolean  datetime  biginteger  bigdecimal  bytearray


## Datos complejos

Apache Pig trabaja con la siguiente jerarquía de relaciones (http://pig.apache.org/docs/r0.17.0/basic.html#relations):

* Una *tuple* es un ser de campos ordenados: (field1, field2, ....).
* Una *bag* es un conjunto de tuplas: {(...), (...), ...}
* Un *map* es un conjunto de parejas [key#value, ....]



### TUPLE

In [0]:
%%writefile data.tsv
A	10	(1, 2)
B	20	(3, 4)
C	30	(5, 6)
D	40	(7, 8)

Writing data.tsv


In [0]:
%%pig
--
-- Los campos del archivo están separados por 
-- tabuladores.
--
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:INT, f3:TUPLE(p:INT, q:INT));
DUMP u;

(A,10,(1,2))
(B,20,(3,4))
(C,30,(5,6))
(D,40,(7,8))


In [0]:
%%pig
--
-- Los campos de la tupla pueden ser accesados
-- por nombre o por posición.
--
r = FOREACH u GENERATE f3.p, f3.$1 ;   
DUMP r;

(1,2)
(3,4)
(5,6)
(7,8)


In [0]:
%%pig
--
-- Aquí se accesan los campos de la tupla por 
-- posición ya que no tienen nombre.
--
u = LOAD 'data.tsv' AS (f1:CHARARRAY, f2:INT, f3:TUPLE(INT, INT));
r = FOREACH u GENERATE $2.$0, $2.$1;
DUMP r;

(1,2)
(3,4)
(5,6)
(7,8)


In [0]:
%%writefile data.tsv
A	(1,  2)	(3,  4)
B	(5,  6)	(7,  8)
C	(9, 10)	(11, 12)

Overwriting data.tsv


In [0]:
%%pig
--
-- Se seleccionan los campos por nombre
--
u = LOAD 'data.tsv'
    AS (f1: CHARARRAY, 
        t1: TUPLE(t1a: INT, t1b: INT), 
        t2: TUPLE(t2a: INT, t2b: INT)); 
r = FOREACH u GENERATE f1, t2.t2b;

### BAG

In [0]:
%%writefile data.tsv
A	10	{( 1,  2),( 3,  4)}
B	20	{( 5,  6),( 7,  8)}
C	30	{( 9, 10),(11, 12)}
D	40	{(13, 14),(15, 16)}

Overwriting data.tsv


In [0]:
%%pig
--
-- Se selecciona el `bag` por nombre
--
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:INT, f3:BAG{t: TUPLE(p:INT, q:INT)});
r = FOREACH u GENERATE f3;
DUMP r;

({(1,2),(3,4)})
({(5,6),(7,8)})
({(9,10),(11,12)})
({(13,14),(15,16)})


In [0]:
%%pig
r = FOREACH u GENERATE f3.p;
DUMP r;

({(1),(3)})
({(5),(7)})
({(9),(11)})
({(13),(15)})


### MAP

In [0]:
%%writefile data.tsv
A	10	[a#1,b#2]
B	20	[a#3,c#4]
C	30	[b#5,c#6]
D	40	[b#7,c#8]

Overwriting data.tsv


In [0]:
%%pig
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:INT, f3:MAP[]);
r = FOREACH u GENERATE f3#'a', f3#'c';
DUMP r

(1,)
(3,4)
(,6)
(,8)


## Manipulación de datos complejos: FLATTEN

In [0]:
%%writefile data.tsv
A	10	(1, 2)
B	20	(3, 4)
C	30	(5, 6)
D	40	(7, 8)

Overwriting data.tsv


In [0]:
%%pig
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:INT, f3:TUPLE(p:INT, q:INT));
DUMP u;

(A,10,(1,2))
(B,20,(3,4))
(C,30,(5,6))
(D,40,(7,8))


In [0]:
%%pig
r = FOREACH u GENERATE f1, FLATTEN(f3);
DUMP r;

(A,1,2)
(B,3,4)
(C,5,6)
(D,7,8)


In [0]:
%%writefile data.tsv
A	10	{(1),(2)}
B	20	{(3),(4)}
C	30	{(5),(6)}
D	40	{(7),(8)}

Overwriting data.tsv


In [0]:
%%pig
u = LOAD 'data.tsv'
    AS (f1:CHARARRAY, f2:INT, f3:BAG{t:(p:INT)});
DUMP u;

(A,10,{(1),(2)})
(B,20,{(3),(4)})
(C,30,{(5),(6)})
(D,40,{(7),(8)})


In [0]:
%%pig
r = FOREACH u GENERATE f1, FLATTEN(f3);
DUMP r;

(A,1)
(A,2)
(B,3)
(B,4)
(C,5)
(C,6)
(D,7)
(D,8)


In [0]:
%%pig
r = FOREACH u GENERATE FLATTEN(f3);
DUMP r;

(1)
(2)
(3)
(4)
(5)
(6)
(7)
(8)


In [0]:
%%pig
--
-- se pueden colocar varios comandos dentro de
-- un FOREACH
--
r1 = FOREACH u {
        GENERATE FLATTEN(f3);
};
DUMP r1;

(1)
(2)
(3)
(4)
(5)
(6)
(7)
(8)


In [0]:
%%pig
r1 = FOREACH u GENERATE (DOUBLE) $1;
DUMP r1;

(10.0)
(20.0)
(30.0)
(40.0)


---

In [0]:
!rm attempt_local* pig_* persons.* data.* demo.py jointable.*

rm: attempt_local*: No such file or directory
rm: persons.*: No such file or directory
rm: demo.py: No such file or directory
rm: jointable.*: No such file or directory
